In [8]:
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)
import random
import numpy as np
import matplotlib.pyplot as plt
from importlib import reload
import subprocess

import mido
import IPython.display as ipd
import pretty_midi
from visual_midi import Plotter
from visual_midi import Preset

import tt_gen
import scales_2

In [86]:
warnings.filterwarnings("ignore", category=DeprecationWarning)

song_name = '12_tone_test'
# generate random prime row
prime_row = random.sample(range(0, 12), 12)
# prime_row = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11]
tt = tt_gen.TwelveTone(prime_row)

tpb = 480  # ticks per beat
mid = mido.MidiFile(ticks_per_beat=tpb)

tempo = mido.bpm2tempo(120) # tempo beats per minute
# Create a separate track (track 0) for tempo changes
tempo_track = mido.MidiTrack()
mid.tracks.append(tempo_track)
tempo_track.append(mido.MetaMessage('set_tempo', tempo=tempo))

track_list = ['bass_1', 'melody_1', 'melody_2']
comb = {c: {} for c in track_list}
# # Load default weights for chromatic scale
# # Makes it easier to edit weights for each track
# octave_weights = np.repeat(scales_2.chromatic['duration_weights'], len(track_list))
# duration_weights = np.repeat(scales_2.chromatic['duration_weights'], len(track_list))
octave_weights = [[0.25,0.5,1,2,1,0.5,0,0,0], [0,0.25,0.50,1,2,1,0.50,0.25,0],
                  [0,0.25,0.5,1,1.5,1,0.75,0.50,0.25]]
duration_weights = [[0,0.1,0.25,0.5,1,0.5,0.5,0.25], [0,0,0.1,0.25,1,0.25,0.1,0],
                    [0.25,0.5,0.75,1,0.5,0.25,0.25,0.1]]
sequence_lengths = [8, 14, 16]    # need longer lengths for tracks playing short notes
sd = [0.24, 0.16, 0.36]
rp = [0.16, 0.16, 0.32]

# add octave and duration weights to comb dict
for i,c in enumerate(comb):
    comb[c]['octave_weights'] = octave_weights[i]
    comb[c]['duration_weights'] = duration_weights[i]
    comb[c]["length"] = sequence_lengths[i]
    comb[c]['sd'] = sd[i]
    comb[c]['rp'] = rp[i]

for i, c in enumerate(comb.values()):
    track = mido.MidiTrack()
    mid.tracks.append(track)
    track.name = track_list[i] # assign the track name
    notes, midi_notes = tt.generator(octave_weights=c['octave_weights'], duration_weights=c['duration_weights'],
                                     length=c["length"], tpb=tpb, 
                                     duration='variable', velocity='variable',
                                     sd=c['sd'], rest_probability=c['rp'])
    # Add the note events to the MIDI track
    for note in midi_notes:
        track.append(note)

# Write the MIDI file to disk
midi_path = f'generated/{song_name}_tracks_{len(comb)}.mid'
mid.save(midi_path)
# Load the MIDI file using pretty_midi
midi_data = pretty_midi.PrettyMIDI(midi_path)
# # Plot the MIDI data
preset = Preset(plot_width=850, plot_height=600)
plotter = Plotter(preset, plot_max_length_bar=16)
plotter.show_notebook(midi_data)
# Synthesize the MIDI file into a waveform
audio_data = midi_data.synthesize(fs=44100)

# Normalize the audio data
audio_data /= np.max(np.abs(audio_data))
# Play the audio data in the Jupyter notebook
ipd.Audio(audio_data, rate=44100)

# # optional - open in GarageBand or DAW of choice
subprocess.call(['open', '-a', 'GarageBand', midi_path])

ValueError: data byte must be in range 0..127